### Supply Chain Mnt Optimization

#### 4 Plants
- Dallas
- Los Angeles
- Seatle
- Denver

#### 4 Stores (Demand)
- Austin
- Sacramento
- Philadelphia
- Boulder

### Costs
- Fixed cost to build plant
- Variable cost to ship product per route

### Research Questions
#### 1) Which plant should we build?
#### 2) Which products should we send on which route?

In [17]:
import pulp

In [28]:
# Costs: Dictionary of Dictionaries

# 4 supply plants
plants = ['Dallas', 'Los Angeles', 'Seatle', 'Denver']

stores = ['Austin', 'Sacramento', 'Philadelphia', 'Boulder']
cost_matrix = [
    [10, 15, 20, 25],  # Затраты для Dallas
    [12, 18, 22, 28],  # Затраты для Los Angeles
    [8, 14, 19, 23],   # Затраты для Seattle
    [11, 16, 21, 27]   # Затраты для Denver
]

In [20]:
costs = {}

for i in range(len(plants)):
    temp_dict = {}
    for j in range(len(stores)):
        temp_dict[stores[j]] = cost_matrix[i][j]
    costs[plants[i]] = temp_dict

In [21]:
# Команда range(len(plants)) создает последовательность чисел от 0 до длины списка plants (не включая это значение).
#
# Если plants — это список с, например, 7 элементами, то range(len(plants)) создаст последовательность чисел от 0 до 6. Эта команда часто используется в циклах for, чтобы итерировать по индексам списка.

for i in range(len(plants)):
    print(plants[i])

Dallas
Los Angeles
Seatle
Denver


In [22]:
temp_dict = {}
for j in range(len(stores)):
    print(stores[j])

Austin
Sacramento
Philadelphia
Boulder


In [26]:
# Routes

routes = []

for p in plants:
    for s in stores:
        routes.append((p, s)) # adding tuple

routes

[('Dallas', 'Austin'),
 ('Dallas', 'Sacramento'),
 ('Dallas', 'Philadelphia'),
 ('Dallas', 'Boulder'),
 ('Los Angeles', 'Austin'),
 ('Los Angeles', 'Sacramento'),
 ('Los Angeles', 'Philadelphia'),
 ('Los Angeles', 'Boulder'),
 ('Seatle', 'Austin'),
 ('Seatle', 'Sacramento'),
 ('Seatle', 'Philadelphia'),
 ('Seatle', 'Boulder'),
 ('Denver', 'Austin'),
 ('Denver', 'Sacramento'),
 ('Denver', 'Philadelphia'),
 ('Denver', 'Boulder')]

### Linear Programming with Pulp for SCM

In [52]:
# product supply
supply = {'Dallas': 900,
          'Los Angeles': 2400,
          'Seatle': 1300,
          'Denver': 1800}

In [53]:
# fixed costs of building each plan
fixedCost = {'Dallas': 75000,
          'Los Angeles': 72000,
          'Seatle': 100000,
          'Denver': 74000}
fixedCost

{'Dallas': 75000, 'Los Angeles': 72000, 'Seatle': 100000, 'Denver': 74000}

In [35]:
# denine a product demand

cost_matrix = [
    #ATX    #SAC    #PHL    #BU
    [10,    15,     20,     25],  # Затраты для Dallas
    [12,    18,     22,     28],  # Затраты для Los Angeles
    [8,     14,     19,     23],   # Затраты для Seattle
    [11,    16,     21,     27]   # Затраты для Denver
]

In [38]:
cost_matrix # a list of lists

[[10, 15, 20, 25], [12, 18, 22, 28], [8, 14, 19, 23], [11, 16, 21, 27]]

In [39]:
# instead of list of lists we create a dictionary of a dictionary
costs

{'Dallas': {'Austin': 10, 'Sacramento': 15, 'Philadelphia': 20, 'Boulder': 25},
 'Los Angeles': {'Austin': 12,
  'Sacramento': 18,
  'Philadelphia': 22,
  'Boulder': 28},
 'Seatle': {'Austin': 8, 'Sacramento': 14, 'Philadelphia': 19, 'Boulder': 23},
 'Denver': {'Austin': 11, 'Sacramento': 16, 'Philadelphia': 21, 'Boulder': 27}}

This code is related to Linear Programming using the PuLP library in Python, which is used for optimization problems. Here’s a breakdown of what the code does:

* `pulp.LpVariable.dicts`: This function creates a dictionary of PuLP LpVariable objects. LpVariable is a class in the PuLP library used to define decision variables in a linear programming problem.

* `'Route'`: This is the name prefix for the variables in the dictionary. Each variable will have this prefix in its name.

* `(plants, stores)`: These are the indices for the dictionary. The resulting dictionary will have keys based on the combinations of plants and stores. If plants and stores are lists or ranges, the dictionary will include an LpVariable for each combination of elements from these lists/ranges.

* `0`: This is the lower bound for the variables, meaning each variable will have a minimum value of 0.

* `None`: This specifies that there is no upper bound on the variables, so they can take any non-negative value.

In [42]:
# Use PULP

route = pulp.LpVariable.dicts('Route', (plants, stores), 0, None, pulp.LpInteger)
route

{'Dallas': {'Austin': Route_Dallas_Austin,
  'Sacramento': Route_Dallas_Sacramento,
  'Philadelphia': Route_Dallas_Philadelphia,
  'Boulder': Route_Dallas_Boulder},
 'Los Angeles': {'Austin': Route_Los_Angeles_Austin,
  'Sacramento': Route_Los_Angeles_Sacramento,
  'Philadelphia': Route_Los_Angeles_Philadelphia,
  'Boulder': Route_Los_Angeles_Boulder},
 'Seatle': {'Austin': Route_Seatle_Austin,
  'Sacramento': Route_Seatle_Sacramento,
  'Philadelphia': Route_Seatle_Philadelphia,
  'Boulder': Route_Seatle_Boulder},
 'Denver': {'Austin': Route_Denver_Austin,
  'Sacramento': Route_Denver_Sacramento,
  'Philadelphia': Route_Denver_Philadelphia,
  'Boulder': Route_Denver_Boulder}}

In [43]:
### pulp.LpVariable.dicts(name, indexs, lowBound, upperBound, category)

In [44]:
build = pulp.LpVariable.dicts('Build_Plant', plants, 0, 1, pulp.LpInteger)

In [45]:
build

{'Dallas': Build_Plant_Dallas,
 'Los Angeles': Build_Plant_Los_Angeles,
 'Seatle': Build_Plant_Seatle,
 'Denver': Build_Plant_Denver}

In [49]:
obj = ""
for (p, s) in routes:
    # print((p, s))
    obj += route[p][s] * costs[p][s]

obj

10*Route_Dallas_Austin + 25*Route_Dallas_Boulder + 20*Route_Dallas_Philadelphia + 15*Route_Dallas_Sacramento + 11*Route_Denver_Austin + 27*Route_Denver_Boulder + 21*Route_Denver_Philadelphia + 16*Route_Denver_Sacramento + 12*Route_Los_Angeles_Austin + 28*Route_Los_Angeles_Boulder + 22*Route_Los_Angeles_Philadelphia + 18*Route_Los_Angeles_Sacramento + 8*Route_Seatle_Austin + 23*Route_Seatle_Boulder + 19*Route_Seatle_Philadelphia + 14*Route_Seatle_Sacramento + 0

In this code snippet, `route[p][s] * costs[p][s]` represents the product of two values in a linear programming problem, and it contributes to the objective function. Here’s what each component typically means:

* `route[p][s]`: This is a decision variable from the PuLP model, where p and s are indices (possibly representing a plant and a store, for example). This variable represents the quantity or decision associated with the route between plant p and store s.

* `costs[p][s]`: This is a cost associated with the route between plant p and store s. It could represent the transportation cost, shipping cost, or any other type of cost relevant to that particular route.

* `route[p][s] * costs[p][s]`: This computes the total cost for the route from plant p to store s by multiplying the quantity of the route (route[p][s]) by the cost associated with that route (costs[p][s]).

* `obj += route[p][s] * costs[p][s]`: This line adds the total cost for each route to the objective function. The variable obj accumulates these costs to form the complete objective function expression.

In [54]:
for p in plants:
    obj += fixedCost[p] * build[p]

In [55]:
obj

75000*Build_Plant_Dallas + 74000*Build_Plant_Denver + 72000*Build_Plant_Los_Angeles + 100000*Build_Plant_Seatle + 10*Route_Dallas_Austin + 25*Route_Dallas_Boulder + 20*Route_Dallas_Philadelphia + 15*Route_Dallas_Sacramento + 11*Route_Denver_Austin + 27*Route_Denver_Boulder + 21*Route_Denver_Philadelphia + 16*Route_Denver_Sacramento + 12*Route_Los_Angeles_Austin + 28*Route_Los_Angeles_Boulder + 22*Route_Los_Angeles_Philadelphia + 18*Route_Los_Angeles_Sacramento + 8*Route_Seatle_Austin + 23*Route_Seatle_Boulder + 19*Route_Seatle_Philadelphia + 14*Route_Seatle_Sacramento + 0

In [57]:
import pandas as pd

# Данные словаря
sales = {
    'North': {'January': 1000, 'February': 1500},
    'South': {'January': 1200, 'February': 1800}
}

# Преобразование в DataFrame
df = pd.DataFrame(sales).T  # .T транспонирует таблицу для правильного представления

# Отображение таблицы
print(df)

       January  February
North     1000      1500
South     1200      1800


In [58]:
sales

{'North': {'January': 1000, 'February': 1500},
 'South': {'January': 1200, 'February': 1800}}

In [60]:
prob = pulp.LpProblem('Supply_Demand', pulp.LpMinimize)
prob += obj, "Total Costs"
prob

Supply_Demand:
MINIMIZE
75000*Build_Plant_Dallas + 74000*Build_Plant_Denver + 72000*Build_Plant_Los_Angeles + 100000*Build_Plant_Seatle + 10*Route_Dallas_Austin + 25*Route_Dallas_Boulder + 20*Route_Dallas_Philadelphia + 15*Route_Dallas_Sacramento + 11*Route_Denver_Austin + 27*Route_Denver_Boulder + 21*Route_Denver_Philadelphia + 16*Route_Denver_Sacramento + 12*Route_Los_Angeles_Austin + 28*Route_Los_Angeles_Boulder + 22*Route_Los_Angeles_Philadelphia + 18*Route_Los_Angeles_Sacramento + 8*Route_Seatle_Austin + 23*Route_Seatle_Boulder + 19*Route_Seatle_Philadelphia + 14*Route_Seatle_Sacramento + 0
VARIABLES
0 <= Build_Plant_Dallas <= 1 Integer
0 <= Build_Plant_Denver <= 1 Integer
0 <= Build_Plant_Los_Angeles <= 1 Integer
0 <= Build_Plant_Seatle <= 1 Integer
0 <= Route_Dallas_Austin Integer
0 <= Route_Dallas_Boulder Integer
0 <= Route_Dallas_Philadelphia Integer
0 <= Route_Dallas_Sacramento Integer
0 <= Route_Denver_Austin Integer
0 <= Route_Denver_Boulder Integer
0 <= Route_Denver_Philad

In [62]:
### supply / demand constraints

for p in plants:
    product_out = ""
    for s in stores:
        product_out += route[p][s]
    prob += product_out <= supply[p] * build[p], "Total product out of plant_" + str(p)

In [63]:
prob

Supply_Demand:
MINIMIZE
75000*Build_Plant_Dallas + 74000*Build_Plant_Denver + 72000*Build_Plant_Los_Angeles + 100000*Build_Plant_Seatle + 10*Route_Dallas_Austin + 25*Route_Dallas_Boulder + 20*Route_Dallas_Philadelphia + 15*Route_Dallas_Sacramento + 11*Route_Denver_Austin + 27*Route_Denver_Boulder + 21*Route_Denver_Philadelphia + 16*Route_Denver_Sacramento + 12*Route_Los_Angeles_Austin + 28*Route_Los_Angeles_Boulder + 22*Route_Los_Angeles_Philadelphia + 18*Route_Los_Angeles_Sacramento + 8*Route_Seatle_Austin + 23*Route_Seatle_Boulder + 19*Route_Seatle_Philadelphia + 14*Route_Seatle_Sacramento + 0
SUBJECT TO
Total_product_out_of_plant_Dallas: - 900 Build_Plant_Dallas
 + Route_Dallas_Austin + Route_Dallas_Boulder + Route_Dallas_Philadelphia
 + Route_Dallas_Sacramento <= 0

Total_product_out_of_plant_Los_Angeles: - 2400 Build_Plant_Los_Angeles
 + Route_Los_Angeles_Austin + Route_Los_Angeles_Boulder
 + Route_Los_Angeles_Philadelphia + Route_Los_Angeles_Sacramento <= 0

Total_product_out_of_

In [65]:
# Demand Constraints

demand = {'Austin': 1700,
          'Sacramento': 1000,
          'Philadelphia':1500,
          'Boulder':1200}

for s in stores:
    product_in = ""
    for p in plants:
        product_in += route[p][s]
    prob += product_in >= demand[s], "Total product store_" + str(s)

In [66]:
prob

Supply_Demand:
MINIMIZE
75000*Build_Plant_Dallas + 74000*Build_Plant_Denver + 72000*Build_Plant_Los_Angeles + 100000*Build_Plant_Seatle + 10*Route_Dallas_Austin + 25*Route_Dallas_Boulder + 20*Route_Dallas_Philadelphia + 15*Route_Dallas_Sacramento + 11*Route_Denver_Austin + 27*Route_Denver_Boulder + 21*Route_Denver_Philadelphia + 16*Route_Denver_Sacramento + 12*Route_Los_Angeles_Austin + 28*Route_Los_Angeles_Boulder + 22*Route_Los_Angeles_Philadelphia + 18*Route_Los_Angeles_Sacramento + 8*Route_Seatle_Austin + 23*Route_Seatle_Boulder + 19*Route_Seatle_Philadelphia + 14*Route_Seatle_Sacramento + 0
SUBJECT TO
Total_product_out_of_plant_Dallas: - 900 Build_Plant_Dallas
 + Route_Dallas_Austin + Route_Dallas_Boulder + Route_Dallas_Philadelphia
 + Route_Dallas_Sacramento <= 0

Total_product_out_of_plant_Los_Angeles: - 2400 Build_Plant_Los_Angeles
 + Route_Los_Angeles_Austin + Route_Los_Angeles_Boulder
 + Route_Los_Angeles_Philadelphia + Route_Los_Angeles_Sacramento <= 0

Total_product_out_of_

In [67]:
### solver!

prob.solve()

1

In [70]:
print('Status: ' , pulp.LpStatus[prob.status])

Status:  Optimal


In [71]:
pulp.LpStatus # dictionary

{0: 'Not Solved',
 1: 'Optimal',
 -1: 'Infeasible',
 -2: 'Unbounded',
 -3: 'Undefined'}

In [72]:
prob.status

1

In [74]:
# taking a look on decision variables to see what plant we should build

for w in prob.variables():
    print(w)

Build_Plant_Dallas
Build_Plant_Denver
Build_Plant_Los_Angeles
Build_Plant_Seatle
Route_Dallas_Austin
Route_Dallas_Boulder
Route_Dallas_Philadelphia
Route_Dallas_Sacramento
Route_Denver_Austin
Route_Denver_Boulder
Route_Denver_Philadelphia
Route_Denver_Sacramento
Route_Los_Angeles_Austin
Route_Los_Angeles_Boulder
Route_Los_Angeles_Philadelphia
Route_Los_Angeles_Sacramento
Route_Seatle_Austin
Route_Seatle_Boulder
Route_Seatle_Philadelphia
Route_Seatle_Sacramento


In [77]:
for w in prob.variables():
    print(w.name, '=', w.varValue)

Build_Plant_Dallas = 0.0
Build_Plant_Denver = 1.0
Build_Plant_Los_Angeles = 1.0
Build_Plant_Seatle = 1.0
Route_Dallas_Austin = 0.0
Route_Dallas_Boulder = 0.0
Route_Dallas_Philadelphia = 0.0
Route_Dallas_Sacramento = 0.0
Route_Denver_Austin = 0.0
Route_Denver_Boulder = 0.0
Route_Denver_Philadelphia = 800.0
Route_Denver_Sacramento = 1000.0
Route_Los_Angeles_Austin = 1600.0
Route_Los_Angeles_Boulder = 0.0
Route_Los_Angeles_Philadelphia = 700.0
Route_Los_Angeles_Sacramento = 0.0
Route_Seatle_Austin = 100.0
Route_Seatle_Boulder = 1200.0
Route_Seatle_Philadelphia = 0.0
Route_Seatle_Sacramento = 0.0


In [80]:
# total expected costs, mln USD

pulp.value(prob.objective)

341800.0